# HW1 Binary Search (BS)

Importing necessary libraries and classes.

In [8]:
# Import required libraries for excecution
from gensim import corpora
import numpy as np
import pandas as pd
from metrics import Metrics
metrics = Metrics()

Load dictionary, document corpus and query corpus.

In [9]:
# Load dictionary, doc_corpus, query_corpus and df with tags
dictionary = corpora.Dictionary.load('vocab.dict')
doc_corpus = corpora.MmCorpus("doc_corpus.mm")
query_corpus = corpora.MmCorpus("query_corpus.mm")
df = pd.read_csv('./data/relevance-judgments.tsv', sep='\t', header=None)
df.columns = ['query', 'doc']

Create binary matrix.

In [10]:
# Create binary matrix
matrix = np.zeros((len(doc_corpus), len(dictionary)), dtype = np.bool_)
for i in range(0, len(doc_corpus)):
    for j in range(0, len(doc_corpus[i])):
        matrix[i, doc_corpus[i][j][0]] = True
np.save('./data/BSmatrix.npy', matrix)

Function to do binary search on a query.

In [11]:
def binary_search(matrix, query):
    """ Returns a matrix with relevant documents based on a query according to binary search method.
    
    Args:
        matrix (numpy.ndarray): binary matrix with document ando word information
        query (list): words contained in the query
    
    Returns:
        list: with relevant documents according to query
    
    """
    
    # Convert query to binary vector
    query_vector = np.zeros((1, len(dictionary)))
    for i in query:
        query_vector[0,i[0]] = True
    
    # Create empty list to gather revelvant documents
    result_list = []
    for i in range(0, len(doc_corpus)):
        
        # Logical AND - OR between matrix and query vector
        if (np.logical_and(query_vector[0], matrix[i, :])).any():
            result_list.append(i + 1)
    return result_list

Funtcion to create evaluation vector.

In [12]:
def convert_to_evaluation(binary_result, query):
    """ Returns a vector in format to use evaluation metrics.
    
    Args:
        binary_result (list): Result to the binary search
        query (int): Number of the query to be compared with
    
    Returns:
        list: relevant documents in format for evaluation metrics
    """
    
    # Empty list to gather relevant document numbers
    tag = []
    for i in df['doc'][query].split(','):
        tag.append(int(i.split(':')[0].split('d')[1]))
        
    # Create the evaluation list
    evaluation = np.zeros(len(binary_result))
    for i in range(0, len(binary_result)):
        if binary_result[i] in tag:
            evaluation[i] = 1
    return evaluation

Create df and tsv files to export results.

In [13]:
""" Runs binary search for wach query

Args:
    query_corpus (gensim.corpora.mmcorpus.MmCorpus): corpus with the queries. 
    df (pandas.core.frame.DataFrame): Dataframe with read queries.
    
Returns:
    df (pandas.core.frame.DataFrame): Dataframe with new column with resulting documents for each query.
"""
df_results = []
for query in query_corpus:
    result_list = ''
    results = binary_search(matrix, query)
    for result in results:
        result_list = result_list + 'd' + str(f'{result:03}') + ','
    df_results.append(result_list[:-1])
df['results'] = df_results
df = df.set_index('query')
# drop column and export tsv file with results
df.drop('doc', axis=1).to_csv('./results/BS-queries_results.tsv', sep='\t', header=False)

In [15]:
convert_to_evaluation(binary_search(matrix, query_corpus[0]))

TypeError: convert_to_evaluation() missing 1 required positional argument: 'query'